<a href="https://colab.research.google.com/github/nazihkalo/Transformers_FinancialNews_Sentiment/blob/master/ktrain_pretrained_models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### A Simplied Interface to Text Classification With Hugging Face Transformers in TensorFlow Using [ktrain](https://github.com/amaiya/ktrain)

*ktrain* requires TensorFlow 2.

In [1]:
!pip3 install -q tensorflow_gpu>=2.0

ERROR: tensorflow 2.2.0rc2 has requirement gast==0.3.3, but you'll have gast 0.2.2 which is incompatible.
ERROR: tensorflow 2.2.0rc2 has requirement tensorboard<2.3.0,>=2.2.0, but you'll have tensorboard 2.1.1 which is incompatible.
ERROR: tensorflow 2.2.0rc2 has requirement tensorflow-estimator<2.3.0,>=2.2.0rc0, but you'll have tensorflow-estimator 2.1.0 which is incompatible.


In [2]:
import tensorflow as tf
print(tf.__version__)

2.1.0


We then need to install *ktrain* library using pip.

In [3]:
!pip3 install -q ktrain

     |████████████████████████████████| 25.2MB 1.9MB/s 
     |████████████████████████████████| 983kB 46.6MB/s 
     |████████████████████████████████| 245kB 37.3MB/s 
     |████████████████████████████████| 552kB 35.4MB/s 
     |████████████████████████████████| 2.8MB 43.5MB/s 
     |████████████████████████████████| 460kB 13.0MB/s 
     |████████████████████████████████| 1.0MB 29.9MB/s 
     |████████████████████████████████| 3.7MB 37.6MB/s 
     |████████████████████████████████| 870kB 39.2MB/s 
ERROR: tensorflow-gpu 2.1.0 has requirement gast==0.2.2, but you'll have gast 0.3.3 which is incompatible.
ERROR: tensorflow-gpu 2.1.0 has requirement tensorboard<2.2.0,>=2.1.0, but you'll have tensorboard 2.2.0 which is incompatible.
ERROR: tensorflow-gpu 2.1.0 has requirement tensorflow-estimator<2.2.0,>=2.1.0rc0, but you'll have tensorflow-estimator 2.2.0rc0 which is incompatible.


### Load a Dataset Into Arrays

In [4]:
from google.colab import files
uploaded = files.upload()

Saving dev.tsv to dev.tsv
Saving train.tsv to train.tsv


In [0]:
import io
import pandas as pd
train_df = pd.read_csv(io.BytesIO(uploaded['train.tsv']), sep ='\t', header = None)
dev_df = pd.read_csv(io.BytesIO(uploaded['dev.tsv']), sep ='\t', header = None)

# Dataset is now stored in a Pandas Dataframe

In [14]:
train_df.columns = ['id', 'label', 'alpha', 'text']
dev_df.columns = ['id', 'label', 'alpha', 'text']
dev_df.head()

,id,label,alpha,text
0,0,1,a,UPDATE 1-Indian shares take negative turn afte...
1,1,0,a,"Traynor Capital Management Inc. Acquires 3,526..."
2,2,0,a,Karani Asset Management LLC Acquires Shares of...
3,3,0,a,Virginia Retirement Systems ET AL Has $67.26 M...
4,4,2,a,Cisco Systems Is Ripe For A Pullback


In [0]:
x_train = train_df.text.reset_index(drop=True)
y_train = train_df.label.astype('category').reset_index(drop=True)
x_test = dev_df.text.reset_index(drop=True)
y_test = dev_df.label.astype('category').reset_index(drop=True)

label_names = ['Abstain', 'Low','High']

## STEP 1:  Preprocess Data and Create a Transformer Model

We will use [DistilBERT](https://arxiv.org/abs/1910.01108).

In [16]:
import ktrain
from ktrain import text
MODEL_NAME = 'distilbert-base-uncased'
t = text.Transformer(MODEL_NAME, maxlen=500, classes=y_test.cat.categories.to_numpy())
trn = t.preprocess_train(x_train, y_train)
val = t.preprocess_test(x_test, y_test)
model = t.get_classifier()
learner = ktrain.get_learner(model, train_data=trn, val_data=val, batch_size=6)

/usr/local/lib/python3.6/dist-packages/ktrain/text/preprocessor.py:367: UserWarning: The class_names argument is replacing the classes argument. Please update your code.
  warnings.warn('The class_names argument is replacing the classes argument. Please update your code.')



preprocessing train...
language: en
train sequence lengths:
	mean : 10
	95percentile : 15
	99percentile : 18


preprocessing test...
language: en
test sequence lengths:
	mean : 10
	95percentile : 15
	99percentile : 20


## STEP 2:  Train the Model

In [17]:
learner.fit_onecycle(5e-5, 4)



begin training using onecycle policy with max lr of 5e-05...
Train for 87 steps, validate for 5 steps
Epoch 1/4
87/87 [==============================] - 73s 841ms/step - loss: 1.0105 - accuracy: 0.5029 - val_loss: 0.9341 - val_accuracy: 0.5496
Epoch 2/4
87/87 [==============================] - 65s 746ms/step - loss: 0.7963 - accuracy: 0.6334 - val_loss: 0.7936 - val_accuracy: 0.6718
Epoch 3/4
87/87 [==============================] - 62s 715ms/step - loss: 0.4927 - accuracy: 0.8292 - val_loss: 0.7446 - val_accuracy: 0.7710
Epoch 4/4
87/87 [==============================] - 62s 711ms/step - loss: 0.2645 - accuracy: 0.9155 - val_loss: 0.7766 - val_accuracy: 0.7481


## STEP 3: Evaluate and Inspect the Model

In [18]:
learner.validate(class_names=t.get_classes())

              precision    recall  f1-score   support

           0       0.94      0.81      0.87        63
           1       0.58      0.90      0.71        40
           2       0.73      0.39      0.51        28

    accuracy                           0.75       131
   macro avg       0.75      0.70      0.70       131
weighted avg       0.79      0.75      0.74       131



array([[51,  9,  3],
       [ 3, 36,  1],
       [ 0, 17, 11]])

Let's examine the validation example about which we were the most wrong.

In [19]:
learner.view_top_losses(n=1, preproc=t)

----------
id:36 | loss:4.45 | true:2 | pred:1)



In [21]:
print(x_test[36], y_test[36])

With Sales In Freefall, Will Samsung Smartphones Go The Way Of BlackBerry And Nokia? 2


This post talks more about computing than `alt.atheism` (the true category), so our model placed it into the only computing category available to it: `comp.graphics`

## STEP 4: Making Predictions on New Data in Deployment

In [0]:
predictor = ktrain.get_predictor(learner.model, preproc=t)

In [27]:
predictor.predict('Shares in the spin-off of South African e-commerce group Naspers sank more than 25% in the first minutes of their market debut in Amsterdam on Wednesday.')

1

In [0]:
# predicted probability scores for each category
predictor.predict_proba('Jesus Christ is the central figure of Christianity.')

array([2.9704000e-03, 5.0002872e-04, 6.5480877e-04, 9.9587470e-01],
      dtype=float32)

In [0]:
predictor.get_classes()

['alt.atheism', 'comp.graphics', 'sci.med', 'soc.religion.christian']

As expected, `soc.religion.christian` is assigned the highest probability.

Let's invoke the `explain` method to see which words contribute most to the classification.

We will need a forked version of the **eli5** library that supportes TensorFlow Keras, so let's install it first.

In [0]:
!pip3 install -q git+https://github.com/amaiya/eli5@tfkeras_0_10_1


In [0]:
predictor.explain('Jesus Christ is the central figure in Christianity.')

The words in the darkest shade of green contribute most to the classification and agree with what you would expect for this example.

We can save and reload our predictor for later deployment.

In [0]:
predictor.save('/tmp/my_distilbert_predictor')

In [0]:
reloaded_predictor = ktrain.load_predictor('/tmp/my_distilbert_predictor')

In [0]:
reloaded_predictor.predict('My computer monitor is really blurry.')

'comp.graphics'